In [4]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim  # TensorFlow-Slim
from sklearn.model_selection import train_test_split

from io import BytesIO
from zipfile import ZipFile
import requests
import os
import time
import math
from PIL import Image
import cv2

In [5]:
def transform_image(img, ang_range, shear_range, trans_range):
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over. 
    
    A Random uniform distribution is used to generate different parameters for transformation
    From blog post: https://nbviewer.jupyter.org/github/vxy10/SCND_notebooks/blob/master/preprocessing_stuff/img_transform_NB.ipynb
    '''
    # Rotation

    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2

    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))
    
    return img

In [6]:
def download_if_not_existing(file_url):
    if not os.path.isfile("data/test.p") or not os.path.isfile("data/train.p"):
        url = requests.get(file_url)
        zipfile = ZipFile(BytesIO(url.content))
        zipfile.extractall(path="data")
        print("data files downloaded and extracted")
    else:
        print("data file exist")

In [7]:
# Load pickled data

# TODO: Fill this in based on where you saved the training and testing data
%time download_if_not_existing("https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip")

print("Loading standard training data")
training_file = "data/train.p"
testing_file = "data/test.p"
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

X_train, y_train = train["features"], train["labels"]
X_test, y_test = test["features"], test["labels"]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

data file exist
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 148 µs
Loading standard training data
X_train shape: (39209, 32, 32, 3)
X_test shape: (12630, 32, 32, 3)


In [9]:
# Now lets use the image jittering we have above to augment the test and validation data sets
# For now we will just add 10 extra images for each image in the training set (before we have split in train/val)

print("Generating and saving augmented data to files")
print("Number of initial training examples =", X_train.shape[0])

augmented_images = []
augmented_labels = []

for i, existing_X in enumerate(X_train):
    augmented_images.append(existing_X)
    augmented_labels.append(y_train[i])
    
    for j in range(5):
        augmented_images.append(transform_image(existing_X, 30, 5, 5))
        augmented_labels.append(y_train[i])
    
X_values = np.array(augmented_images, dtype=np.float32())
y_values = np.array(augmented_labels, dtype=np.float32())

with open("data/augmented_X_train.p", "wb") as f:
    pickle.dump(X_values, f, -1)

with open("data/augmented_y_train.p", "wb") as f:
    pickle.dump(y_values, f, -1)

print("Number of augmented training examples =", X_values.shape[0])
print("Saved to /data directory. Re-run with USE_AUGMENTED_DATA = True")

Generating and saving augmented data to files
Number of initial training examples = 39209


MemoryError: 